<a href="https://colab.research.google.com/github/AshStringer/FDSFE_AStringer/blob/main/HW7/Kaggle_Customer_Profile_DBscan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np


In [2]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
pwd

'/content'

In [4]:
cd /content/gdrive/MyDrive/Kaggle

/content/gdrive/MyDrive/Kaggle


In [5]:
pip install kaggle

In [6]:
ls kaggle.json

kaggle.json


first make an account on kaggle and get your kaggle.json file from Account -> Settings. Then put it in a folder on your google drive called `kaggle`


In [7]:
!chmod 600 kaggle.json #only do it once!

In [8]:
import json
import os
envs = json.load(open("kaggle.json", "r"))
os.environ["KAGGLE_USERNAME"] = envs['username']
os.environ["KAGGLE_KEY"] = envs['key']

In [9]:
import kaggle


# get info on how to use kaggle datasets to find datasets by name


In [10]:
!kaggle datasets --help

usage: kaggle datasets [-h] {list,files,download,create,version,init,metadata,status} ...

options:
  -h, --help            show this help message and exit

commands:
  {list,files,download,create,version,init,metadata,status}
    list                List available datasets
    files               List dataset files
    download            Download dataset files
    create              Create a new dataset
    version             Create a new dataset version
    init                Initialize metadata file for dataset creation
    metadata            Download metadata about a dataset
    status              Get the creation status for a dataset


# find a file named customer personality analysis

In [11]:


!kaggle datasets list -s "customer"

ref                                                      title                                  size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------  ------------------------------------  -----  -------------------  -------------  ---------  ---------------  
imakash3011/customer-personality-analysis                Customer Personality Analysis          62KB  2021-08-22 18:15:05         160303       2556  0.9705882        
blastchar/telco-customer-churn                           Telco Customer Churn                  172KB  2018-02-23 18:20:00         280190       2847  0.88235295       
datascientistanna/customers-dataset                      Shop Customer Data                     23KB  2023-02-07 18:42:21          39206        534  1.0              
vetrirah/customer                                        Customer Segmentation                 105KB  2020-08-28 11:19:39          18138        162  1.0             

In [12]:
cd /content/gdrive/MyDrive/Kaggle

/content/gdrive/MyDrive/Kaggle


In [13]:
!mkdir kaggledatasets

mkdir: cannot create directory ‘kaggledatasets’: File exists


In [14]:
!cd /content/gdrive/MyDrive/Kaggle/kaggledatasets

In [15]:
!kaggle datasets download -d imakash3011/customer-personality-analysis


Dataset URL: https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis
License(s): CC0-1.0
customer-personality-analysis.zip: Skipping, found more recently modified local copy (use --force to force download)


# unpack the file

In [16]:
!ls

customer-personality-analysis.zip  kaggledatasets  kaggle.json	marketing_campaign.csv


In [ ]:
!unzip customer-personality-analysis.zip

Archive:  customer-personality-analysis.zip
replace marketing_campaign.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# read in the file

In [ ]:
import pandas as pd
customers = pd.read_csv("marketing_campaign.csv", sep="\t")
customers

# remove all columns that do not contain numbers (or equivalently select all columns that do)

In [ ]:
customers = customers.select_dtypes(include="number")
customers

In [ ]:
customers.info()

In [ ]:
column_length = {c:len(customers[c].unique()) for c in customers.columns}
column_length

# remove all columns that have less than 50 unique values (or select all those that have >50)

In [ ]:
customers = customers[[c for c in customers.columns if column_length[c] > 50]]

In [ ]:
customers

# remove the ID which should be a mute index

In [ ]:
customers.drop("ID", axis=1, inplace=True)

In [ ]:
customers

# create a column that contains the age of the customer, instead of the birth year

In [ ]:
customers["Age"] = 2024 - customers["Year_Birth"]
customers.drop("Year_Birth", axis=1, inplace=True)
customers

# scale the data so each feature is mean 0 and standard deviation 1

In [ ]:
from sklearn.preprocessing import StandardScaler


In [ ]:
scaler = StandardScaler()
scaled_customers = scaler.fit_transform(customers)
scaled_customers = pd.DataFrame(scaled_customers, columns=customers.columns)
scaled_customers

In [ ]:
#check that each column is mean 0 stdev 1
scaled_customers.describe()

# PLOT HISTOGRAMS OF ALL VARIABLES

In [ ]:


import matplotlib.pyplot as plt


for column in scaled_customers.columns:
    plt.figure(figsize=(8, 6))  # Adjust figure size as needed
    plt.hist(scaled_customers[column], bins=30)  # Adjust number of bins as needed
    plt.title(f'Histogram of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()

This is a histogram plot of all the variables in the data.

# write a caption for the scatter matrix below

In [ ]:
# prompt: plot a scatter matrix of the scaled dataset

import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

# Assuming 'scaled_customers' DataFrame is already created

# Create a scatter matrix plot
scatter_matrix(scaled_customers, alpha=0.2, figsize=(15, 15), diagonal='kde')
plt.show()

This scatter matrix is very interesting. Income plots look different than the others. Also, the plot is very linear.

# fix NaN by IMPUTING WITH k-NN

In [ ]:


from sklearn.impute import KNNImputer


imputer = KNNImputer(n_neighbors=5)

scaled_customers_imputed = imputer.fit_transform(scaled_customers)

scaled_customers_imputed = pd.DataFrame(scaled_customers_imputed, columns=scaled_customers.columns)

In [ ]:
scaled_customers.info()

# CLUSTER THE DATA WITH DB SCAN, THEN SEE HOW MANY CLUSTERS THERE ARE AND HOW MANY OBJECTS IN EACH CLUSTER (your result may differe from mine!)

In [ ]:

from sklearn.cluster import DBSCAN


dbscan = DBSCAN(eps=0.3, min_samples=5)
clusters = dbscan.fit_predict(scaled_customers_imputed)


unique_clusters, counts = np.unique(clusters, return_counts=True)


print("Number of clusters:", len(unique_clusters) - (1 if -1 in unique_clusters else 0))

for cluster, count in zip(unique_clusters, counts):
    print(f"Cluster {cluster}: {count} objects")

I chose to use eps=0.3 so that I could get smaller clusters. So, that more noise points could be marked as outliers and to not miss any important clusters.

This is a cluster plot of the data which is unique because you can see different groups which might look different than a normal plot.

In [ ]:
# prompt: plot a box and whiskers plot of the distribution of scaled_customer features for each cluster

import matplotlib.pyplot as plt

# Assuming 'scaled_customers' DataFrame and 'clusters' are already created

# Create a box and whiskers plot for each cluster
for column in scaled_customers.columns:
  if column != 'cluster':  # Exclude the cluster column itself
    plt.figure(figsize=(8, 6))
    scaled_customers.boxplot(column=column, by='cluster')
    plt.title(f'Boxplot of {column} by Cluster')
    plt.suptitle('')  # Remove the default suptitle
    plt.xlabel('Cluster')
    plt.ylabel(column)
    plt.show()

This is also an interesting way to show the data by using box and whisker plots.

In [ ]:
# prompt: produce a tsne 2d projection of scaled_data and plot it colored by cluster

from sklearn.manifold import TSNE

# Assuming 'scaled_customers' DataFrame with 'cluster' column is already created

# Create a t-SNE object
tsne = TSNE(n_components=2, random_state=42, early_exaggeration=15, perplexity=50)

# Fit and transform the scaled data to reduce it to 2 dimensions
tsne_results = tsne.fit_transform(scaled_customers.drop('cluster', axis=1))

# Add the t-SNE results to the DataFrame
scaled_customers['tsne_x'] = tsne_results[:, 0]
scaled_customers['tsne_y'] = tsne_results[:, 1]

# Plot the t-SNE projection, colored by cluster
plt.figure(figsize=(10, 8))
plt.scatter(scaled_customers['tsne_x'], scaled_customers['tsne_y'], c=scaled_customers['cluster'], cmap='viridis')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('t-SNE Projection of Customer Data Colored by Cluster')
plt.colorbar()
plt.show()

This just used colors to distinguish each cluster which is very helpful when looking at it.